In [2]:
import pandas as pd
import numpy as np

In [3]:
system_df = pd.read_excel('./결의서전표.xlsx')
system_df = system_df.loc[ :system_df.shape[0]-2, ['결의일자-번호','수입금액','지출금액']]
system_df['결의일자'] = [string[:-2] for string in system_df['결의일자-번호']]
system_df = system_df.loc[ : , ['결의일자','수입금액','지출금액']]

system_df = system_df.sort_values('결의일자').reset_index(drop=True)
system_df.tail()

,결의일자,수입금액,지출금액
84,2019/12/26,40000,0
85,2019/12/28,0,49000
86,2019/12/29,0,10
87,2019/12/29,100,0
88,2019/12/31,10000,0


In [4]:
account_df = pd.read_excel('./통장거래내역.xls', skiprows = 9)[['거래일자','출금금액(원)','입금금액(원)']]
account_df['출금금액(원)'] = account_df['출금금액(원)'].fillna(0).astype(int)
account_df['입금금액(원)'] = account_df['입금금액(원)'].fillna(0).astype(int)
account_df = account_df.sort_values('거래일자').reset_index(drop=True)
account_df.tail()

,거래일자,출금금액(원),입금금액(원)
108,2019/12/28,31000,0
109,2019/12/28,18000,0
110,2019/12/29,0,100
111,2019/12/29,10,0
112,2019/12/31,0,10000


In [4]:
income_check = sum(system_df.수입금액) == sum(account_df['입금금액(원)'])
income_check

True

In [5]:
cost_check = sum(system_df.지출금액) == sum(account_df['출금금액(원)'])
cost_check

True

In [5]:
system_df_groupby = system_df.groupby('결의일자')[['수입금액','지출금액']].sum()
system_df_groupby.tail()

,수입금액,지출금액
결의일자,,
2019/12/23,10000,140792
2019/12/26,40000,0
2019/12/28,0,49000
2019/12/29,100,10
2019/12/31,10000,0


In [6]:
account_df_groupby = account_df.groupby('거래일자')[['입금금액(원)','출금금액(원)']].sum()
account_df_groupby.tail()

,입금금액(원),출금금액(원)
거래일자,,
2019/12/23,10000,140792
2019/12/26,40000,0
2019/12/28,0,49000
2019/12/29,100,10
2019/12/31,10000,0


In [8]:
(system_df_groupby.shape, account_df_groupby.shape)

((78, 2), (77, 2))

In [9]:
[idx for idx in system_df_groupby.index if idx not in account_df_groupby.index]

['2019/01/10']

In [10]:
[idx for idx in account_df_groupby.index if idx not in system_df_groupby.index]

[]

In [11]:
[ (idx, item1, item2) for (idx, item1, item2) 
     in zip(system_df_groupby.index[3:], system_df_groupby['수입금액'][3:], account_df_groupby['입금금액(원)'][2:])
     if item1 != item2 ]

[('2019/02/25', 90000, 100000)]

## 결론 : 양쪽 총액은 맞지만, 세부내역에서 차이가 있음
1. 결의서전표 2019/01/10-3 날짜 수입 10,000 원(개인후원)은 통장 거래내역에 존재하지 않음
2. 결의서전표 2019/02/25-4 날짜 수입 90,000 원(개인후원 등)은 통장 거래내역보다 10,000 원 적음
3. 우연히 두 오류가 동시에 발생하면서, 총액 기준으로는 문제가 없어 보였던 것임

In [9]:
merge_df = system_df_groupby.merge(account_df_groupby, how='outer',left_on=system_df_groupby.index, right_on=account_df_groupby.index)
merge_df = merge_df.rename(columns={'key_0':'날짜'}).set_index('날짜')
merge_df = merge_df.fillna(0).astype(int)
merge_df.head()

,수입금액,지출금액,입금금액(원),출금금액(원)
날짜,,,,
2019/01/04,0,69200,0,69200
2019/01/09,0,66000,0,66000
2019/01/10,10000,0,0,0
2019/01/18,0,400000,0,400000
2019/01/21,10000,0,10000,0


In [13]:
unmatch_income = [ (idx, item1, item2) for (idx, item1, item2) 
     in zip(merge_df.index, merge_df['수입금액'], merge_df['입금금액(원)'])
     if item1 != item2 ]

if not unmatch_income:
    print('[수입 결의서] 정상입니다.')
else:
    print('====== [수입 결의서] 문제가 발견되었습니다. =====\n')
    for idx, line in enumerate(unmatch_income):
        print(f'{idx+1}. [수입 결의서] {line[0]} 날짜에 {line[1]}원으로 입력된 부분은 통장 거래내역에 {line[2]}원입니다.\n')

====== [수입 결의서] 문제가 발견되었습니다. =====

1. [수입 결의서] 2019/01/10 날짜에 10000원으로 입력된 부분은 통장 거래내역에 0원입니다.

2. [수입 결의서] 2019/02/25 날짜에 90000원으로 입력된 부분은 통장 거래내역에 100000원입니다.



In [14]:
unmatch_cost = [ (idx, item1, item2) for (idx, item1, item2) 
     in zip(merge_df.index, merge_df['지출금액'], merge_df['출금금액(원)'])
     if item1 != item2 ]

if not unmatch_cost:
    print('[지출 결의서] 정상입니다.')
else:
    print('====== [지출 결의서] 문제가 발견되었습니다. =====\n')
    for idx, line in enumerate(unmatch_cost):
        print(f'{idx+1}. [지출 결의서] {line[0]} 날짜에 {line[1]}원으로 입력된 부분은 통장 거래내역에 {line[2]}원입니다.\n')

[지출 결의서] 정상입니다.
